In [1]:
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [4]:
kvs = osm.toolkit.get_osm_kvs()

In [5]:
tags = Tags(kvs)

In [107]:
area_Glasgow = "(55.82, -4.34, 55.9, -4.22)"
area_London = "(51.3426, -0.5527, 51.6726, 0.1929)"
area_Moscow = "(55.503, 37.0789,56.003, 38.1871)"
area_Budapest = "(47.4439, 18.9315, 47.5851, 19.1711)"
area_Paris = "(48.744, 2.1265, 49.0316, 2.6099)"

In [7]:
df_stations = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.station,area=area_Glasgow)

In [8]:
df_stations["grouping"] = df_stations["network"]

In [9]:
df_stations['grouping'] = df_stations['grouping'].str.split(";",expand=True)[0]

In [10]:
df_stations = df_stations.dropna(subset=['grouping'])

In [11]:
output_file("Glasgow_Stations.html")

tooltips = {"tooltips":[("id", "@id"),("key", "@key"),("tag", "@tag"),("name", "@name"),("network", "@network"),("operator", "@operator"),("(lat,lon)", "@coordinates")]}

p = osm.toolkit.plot_latlon(df_stations,kwargs_for_figure=tooltips,kwargs_for_marker={"legend_group":"grouping"})

p.legend.visible = False

show(p)

In [12]:
df_station_subway = osm.toolkit.get_osm_data(key=tags.station,tag=tags.station.subway,area=area_Glasgow)


In [13]:
df_railway_subway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.subway,area=area_Glasgow,output='geom')


In [14]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    
# create a plot of the subway system    
output_file("Glasgow_subway.html")

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}
    
p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),("key", "@key"),("tag", "@tag"),("name", "@name"),("network", "@network"),("operator", "@operator"),("(lat,lon)", "@coordinates")]


# plot the railway lines
for row in df_railway_subway.iterrows():
    way=pd.DataFrame.from_records(row[1]['geometry'])
       
    x,y = TRANSFORM_4326_TO_3857.transform(way['lat'],way['lon'])   
    
    p.line(x=x,y=y,line_width=2,line_color='orange',legend_label=row[1]['name'])
    
    
    
    
# plot the stations and add a hover tool
map_df = df_station_subway[['id','lat','lon','key','tags','tag','name','network','operator']].copy()

map_df["x"],map_df["y"] = TRANSFORM_4326_TO_3857.transform(map_df['lat'],map_df['lon'])  
map_df["coordinates"]=tuple(zip(map_df["lat"],map_df["lon"]))
map_df['tags'] = map_df['tags'].apply(str)
map_df['tag'] = map_df['tag'].apply(str)
map_df['key'] = map_df['key'].apply(str)


source = ColumnDataSource(map_df)


for legend_group in set(map_df["network"]):
    
    stations = p.scatter(x="x", y="y",
             source=source,
             size=14,
             line_width=3,
             fill_color = "white",
             line_color='orange',
             legend_label=legend_group)

    p.add_tools(HoverTool(tooltips=tooltips, renderers=[stations]))
    
    
show(p)

In [15]:
df_station_subway = osm.toolkit.get_osm_data(key=tags.station,tag=tags.station.subway,area=area_London)


In [16]:
df_railway_subway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.subway,area=area_London,output='geom')


In [18]:
df_railway_subway['line'] = df_railway_subway['line'].fillna('Service')

df_railway_subway['first_line'] = df_railway_subway['line'].apply(lambda x: str(x).split(';')[0])

In [21]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    
# create a plot of the subway system    
output_file("London_subway.html")

color_mapping = {'Central':'red',
                 'Piccadilly':'navy',
                 'Northern':'black',
                 'Victoria':'skyblue',
                 'Metropolitan':'purple',
                 'District':'green',
                 'Circle':'yellow',
                 'Bakerloo':'brown',
                 'Jubilee':'silver',
                 'Hammersmith & City':'pink',
                 'Northern City':'black',
                 'Overground':'orange',
                 'Waterloo & City':'teal',
                 'Service':'white',
                 'Other':'white',
                }

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}
    
p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),
            ("key", "@key"),
            ("tag", "@tag"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]


# plot the railway lines    
for line in set(df_railway_subway['line']):

    #print(line)
    
    df_lines = df_railway_subway.loc[df_railway_subway['line']==line]
  
    xx = list()
    yy = list()
    dashes = list()
    
    railway_lines = line.replace(',', ';').split(';')
    
    for row in df_lines['geometry']:
    
        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(row)['lat']),
                                                 list(pd.DataFrame.from_records(row)['lon']))
    
        xx.append(xxx)
        yy.append(yyy)
        
        if len(xxx)%2 == 0:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2))
        else:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2-0.5)+[5])
            
    
    source = ColumnDataSource({'xs':xx,'ys':yy,'line_dash':dashes})
    
    if len(railway_lines) == 1:
        line_dash = 'solid'
    else:
        line_dash = 'line_dash'
             
    for cnt,railway_line in enumerate(railway_lines):

            railway_line = railway_line.strip()

            p.multi_line('xs',
                   'ys',
                   source=source,
                   line_width=2,
                   legend_label=str(railway_line),
                   line_color = color_mapping[railway_line],
                   line_dash = line_dash,
                   line_dash_offset = cnt*5
                  )

    
    
# plot the stations and add a hover tool
map_df = df_station_subway[['id','lat','lon','key','tags','tag','name','line','network','operator']].copy()

map_df["x"],map_df["y"] = TRANSFORM_4326_TO_3857.transform(map_df['lat'],map_df['lon'])  
map_df["coordinates"]=tuple(zip(map_df["lat"],map_df["lon"]))
map_df['tags'] = map_df['tags'].apply(str)
map_df['tag'] = map_df['tag'].apply(str)
map_df['key'] = map_df['key'].apply(str)
map_df['line'] = map_df['line'].fillna('Other')

stations = list()
for legend_group in set(map_df["line"]):
    
    source = ColumnDataSource(map_df.loc[map_df["line"]==legend_group])

    
    station_lines = legend_group.split(';')
        
    for cnt,station_line in enumerate(station_lines):
        
        if len(station_lines)>1:
            
            station = p.wedge(x="x", y="y",
                            source=source,
                            start_angle=(cnt-1)/len(station_lines)*2*3.142,
                            end_angle=cnt/len(station_lines)*2*3.1412,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        else:
        
            station = p.circle(x="x", y="y",
                            source=source,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        stations.append(station)
    

p.add_tools(HoverTool(tooltips=tooltips, renderers=stations))

p.legend.click_policy="hide"
    
    
show(p)

In [23]:
df_station_subway = osm.toolkit.get_osm_data(key=tags.station,tag=tags.station.subway,area=area_Moscow)


In [24]:
df_railway_subway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.subway,area=area_Moscow,output='geom')


In [26]:
df_railway_subway['line'] = df_railway_subway['name'].fillna('Service')

df_railway_subway['first_line'] = df_railway_subway['line'].apply(lambda x: str(x).split(';')[0])

In [53]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    
# create a plot of the subway system    
output_file("Moscow_subway.html")

color_mapping = {'Кольцевая линия':'brown',
            'Некрасовская линия':'pink',
            'Ответвление ТКЛ-КожЛ':'purple',
            'Калининская линия':'yellow',
            'Бутовская линия':'lightblue',
            'Служебная соединительная ветвь':'white',
            '3-6':'gray',
            'Каховская линия':'black',
            'Калужско-Рижская линия':'orange',
            'Большая кольцевая линия':'#82C0C0',
            'Люблинско-Дмитровская линия':'#b4d445',
            'Филёвская линия':'lightblue',
            'Сокольническая линия':'red',
            'Серпуховско-Тимирязевская линия':'gray',
            'Арбатско-Покровская линия':'blue',
            'Третий станционный путь':'violet',
            'Третий пересадочный контур':'darkturquoise',
            'БКЛ':'#82C0C0',
            'Замоскворецкая линия':'darkgreen',
            'Калининско-Солнцевская линия':'yellow',
            'Таганско-Краснопресненская линия':'violet',
            '4-5':'gray',
            'Солнцевская линия':'yellow',
            '21':'red',
            'Service':'white',
            'Other':'white',
                }

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}
    
p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),
            ("key", "@key"),
            ("tag", "@tag"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]


# plot the railway lines    
for line in set(df_railway_subway['line']):

    #print(line)
    
    df_lines = df_railway_subway.loc[df_railway_subway['line']==line]
  
    xx = list()
    yy = list()
    dashes = list()

    railway_lines = line.replace(',', ';').split(';')
    
    for row in df_lines['geometry']:
    
        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(row)['lat']),
                                                 list(pd.DataFrame.from_records(row)['lon']))
    
        xx.append(xxx)
        yy.append(yyy)
        
        
        if len(xxx)%2 == 0:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2))
        else:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2-0.5)+[5])
            
    
    source = ColumnDataSource({'xs':xx,'ys':yy,'line_dash':dashes})
    
    if len(railway_lines) == 1:
        line_dash = 'solid'
    else:
        line_dash = 'line_dash'
             
    for cnt,railway_line in enumerate(railway_lines):

            railway_line = railway_line.strip()

            p.multi_line('xs',
                   'ys',
                   source=source,
                   line_width=2,
                   legend_label=str(railway_line),
                   line_color = color_mapping[railway_line],
                   line_dash = line_dash,
                   line_dash_offset = cnt*5
                  )

    
    
# plot the stations and add a hover tool
map_df = df_station_subway[['id','lat','lon','key','tags','tag','name','colour','network','operator']].copy()

map_df["x"],map_df["y"] = TRANSFORM_4326_TO_3857.transform(map_df['lat'],map_df['lon'])  
map_df["coordinates"]=tuple(zip(map_df["lat"],map_df["lon"]))
map_df['tags'] = map_df['tags'].apply(str)
map_df['tag'] = map_df['tag'].apply(str)
map_df['key'] = map_df['key'].apply(str)
map_df['line'] = map_df['colour'].fillna('Other')

stations = list()
for legend_group in set(map_df["line"]):
    
    source = ColumnDataSource(map_df.loc[map_df["line"]==legend_group])

    
    station_lines = legend_group.split(';')
        
    for cnt,station_line in enumerate(station_lines):
        
        if len(station_lines)>1:
            
            station = p.wedge(x="x", y="y",
                            source=source,
                            start_angle=(cnt-1)/len(station_lines)*2*3.142,
                            end_angle=cnt/len(station_lines)*2*3.1412,
                            radius=100,
                            line_width=1,
                            fill_color = station_line,
                            line_color = 'white',
                            #legend_label=station_line
                            )
            
        else:
        
            station = p.circle(x="x", y="y",
                            source=source,
                            radius=100,
                            line_width=1,
                            fill_color = station_line,
                            line_color = 'white',
                            #legend_label=station_line
                            )
            
        stations.append(station)
    

p.add_tools(HoverTool(tooltips=tooltips, renderers=stations))

p.legend.click_policy="hide"
    
    
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "fill_color" value "Other" [renderer: GlyphRenderer(id='64235', ...)]


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "fill_color" value "Other" [renderer: GlyphRenderer(id='64235', ...)]


In [98]:
df_station_subway = osm.toolkit.get_osm_data(key=tags.station,tag=tags.station.subway,area=area_Budapest)


In [99]:
df_railway_subway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.subway,area=area_Budapest,output='geom')


In [100]:
df_railway_subway['line'] = df_railway_subway['name'].fillna('Service')

df_railway_subway['first_line'] = df_railway_subway['line'].apply(lambda x: str(x).split(';')[0])

In [101]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    
# create a plot of the subway system    
output_file("Budapest_subway.html")

color_mapping = {'M1':'yellow',
                 'M2':'red',
                 'M3':'blue',
                 'M4':'green',
            'Service':'white',
            'Other':'white',
                }

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}
    
p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),
            ("key", "@key"),
            ("tag", "@tag"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]


# plot the railway lines    
for line in set(df_railway_subway['line']):

    #print(line)
    
    df_lines = df_railway_subway.loc[df_railway_subway['line']==line]
  
    xx = list()
    yy = list()
    dashes = list()

    railway_lines = line.replace(',', ';').split(';')
    
    for row in df_lines['geometry']:
    
        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(row)['lat']),
                                                 list(pd.DataFrame.from_records(row)['lon']))
    
        xx.append(xxx)
        yy.append(yyy)
        
        
        if len(xxx)%2 == 0:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2))
        else:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2-0.5)+[5])
            
    
    source = ColumnDataSource({'xs':xx,'ys':yy,'line_dash':dashes})
    
    if len(railway_lines) == 1:
        line_dash = 'solid'
    else:
        line_dash = 'line_dash'
             
    for cnt,railway_line in enumerate(railway_lines):

            railway_line = railway_line.strip()

            p.multi_line('xs',
                   'ys',
                   source=source,
                   line_width=2,
                   #legend_label=str(railway_line),
                   line_color = color_mapping[railway_line],
                   line_dash = line_dash,
                   line_dash_offset = cnt*5
                  )

    
    
# plot the stations and add a hover tool
map_df = df_station_subway[['id','lat','lon','key','tags','tag','name','line','colour','operator']].copy()

map_df["x"],map_df["y"] = TRANSFORM_4326_TO_3857.transform(map_df['lat'],map_df['lon'])  
map_df["coordinates"]=tuple(zip(map_df["lat"],map_df["lon"]))
map_df['tags'] = map_df['tags'].apply(str)
map_df['tag'] = map_df['tag'].apply(str)
map_df['key'] = map_df['key'].apply(str)
map_df['line'] = map_df['line'].fillna('Other')

stations = list()
for legend_group in set(map_df["line"]):
    
    source = ColumnDataSource(map_df.loc[map_df["line"]==legend_group])

    
    station_lines = legend_group.split(';')
        
    for cnt,station_line in enumerate(station_lines):
        
        if len(station_lines)>1:
            
            station = p.wedge(x="x", y="y",
                            source=source,
                            start_angle=(cnt-1)/len(station_lines)*2*3.142,
                            end_angle=cnt/len(station_lines)*2*3.1412,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        else:
        
            station = p.circle(x="x", y="y",
                            source=source,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        stations.append(station)
    

p.add_tools(HoverTool(tooltips=tooltips, renderers=stations))

p.legend.click_policy="hide"
    
    
show(p)

In [135]:
df_metro = osm.toolkit.get_osm_data(key=tags.name__en,tag="Paris Metro",area="")

In [143]:
df_metro_lines = pd.DataFrame.from_records(df_metro['members'][0])

In [159]:
df_metro_line = osm.toolkit.get_osm_data_by_id(member_id=df_metro_lines.iloc[0]['ref'],obj_type="rel",output="center")

In [193]:
df_metro_routes = pd.DataFrame.from_records(df_metro_line['members'][0])

In [207]:
df_metro_routes

,type,ref,role
0,relation,123784,
1,relation,3328692,


In [194]:
df_metro_route = osm.toolkit.get_osm_data_by_id(member_id=df_metro_routes.iloc[0]['ref'],obj_type="rel",output="center")

In [195]:
df_route_members=pd.DataFrame.from_records(df_metro_route['members'][0])

df_railway_ids = list(df_route_members[df_route_members['role']=='']['ref'])

In [201]:
df_railways = pd.DataFrame()

In [204]:
df_railways = pd.concat([df_railway,osm.toolkit.get_osm_data_by_id(member_id=df_railway_ids[0],obj_type="way",output="geom")])

In [213]:
df_railways = pd.DataFrame()

df_metro = osm.toolkit.get_osm_data(key=tags.name__en,tag="Paris Metro",area="")

df_metro_lines = pd.DataFrame.from_records(df_metro['members'][0])

for metro_line_id in df_metro_lines['ref']:
    #print(metro_line)
    df_metro_line = osm.toolkit.get_osm_data_by_id(member_id=metro_line_id,obj_type="rel")
    
    df_metro_routes = pd.DataFrame.from_records(df_metro_line['members'][0])
    
    for route in df_metro_routes['ref']:
        
        df_metro_route = osm.toolkit.get_osm_data_by_id(member_id=route,obj_type="rel",output="geom")
    
        df_railways = pd.concat([df_railways,df_metro_route])
        
        #print(df_railways)
    

    type      id                                             bounds  \
0  route  123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   

                                             members  \
0  [{'type': 'node', 'ref': 241928757, 'role': 's...   

                                                tags air_conditioning  \
0  {'air_conditioning': 'yes', 'bicycle:condition...              yes   

      bicycle:conditional   colour                  from    line  ...  \
0  yes @ (Su 05:30-16:30)  #F2C931  Château de Vincennes  subway  ...   

     ref:FR:RATP ref:structurae   route short_name  \
0  1001100010001       10000240  subway         M1   

                                              source          to      twitter  \
0  http://www.ratp.fr/fr/ratp/r_102432/cyclistes-...  La Défense  Ligne1_RATP   

  wheelchair wikidata                     wikipedia  
0         no   Q13224  fr:Ligne 1 du métro de Paris  

[1 rows x 25 columns]
    type       id                              

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   

                                             members  \
0  [{'type': 'node', 'ref': 241928757, 'role': 's...   
0  [{'type': 'node', 'ref': 1953056975, 'role': '...   
0  [{'type': 'node', 'ref': 243496729, 'role': 's...   
0  [{'type': 'node', 'ref': 4317605697, 'role': '...   
0  [{'type': 'node', 'ref': 7658079724, 'role': '...   
0  [{'type': 'node', 'ref': 255161921, 'role': 's...   

                                                tags air_conditioning  \
0  {'air_conditioning': 'yes'

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   

                                             members  \
0  [{'type': 'node', 'ref': 241928757, 'role': 's...   
0  [{'type': 'node', 'ref': 1953056975, 'role': '...   
0  [{'type': 'node', 'ref': 243496729, 'role': 's...   
0  [{'type': 'node', 'ref': 4317605697, 'role': '...   

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   

                                             members  \
0  [{'type': 'node', 'ref': 241928757, 'role': 's...   
0  [{'type': 'node', 'r

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 48.829542, 'minlon': 2.284708, 'ma

...

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 48.829542, 'minlon': 2.284708, 'ma

No specified nodes found in area
    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 4

No specified nodes found in area
    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 4

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 48.829542, 'minlon': 2.284708, 'ma

    type       id                                             bounds  \
0  route   123784  {'minlat': 48.8445799, 'minlon': 2.2299577, 'm...   
0  route  3328692  {'minlat': 48.8444578, 'minlon': 2.2299297, 'm...   
0  route  4728208  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route   123917  {'minlat': 48.8479623, 'minlon': 2.2777713, 'm...   
0  route  7731250  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route   123789  {'minlat': 48.8629423, 'minlon': 2.2803973, 'm...   
0  route  5456573  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123802  {'minlat': 48.8651235, 'minlon': 2.3985462, 'm...   
0  route   123767  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  3326844  {'minlat': 48.8166247, 'minlon': 2.3182394, 'm...   
0  route  4730854  {'minlat': 48.8309756, 'minlon': 2.3553608, 'm...   
0  route   123674  {'minlat': 48.8309756, 'minlon': 2.3553135, 'm...   
0  route   123912  {'minlat': 48.829542, 'minlon': 2.284708, 'ma

In [230]:
df_railways = df_railways.reset_index()

In [231]:
df_metro_data

,index,type,id,bounds,members,tags,air_conditioning,bicycle:conditional,colour,from,...,twitter,wheelchair,wikidata,wikipedia,stops,source:colour,railway:track_ref,start_date,via,old_name
0,0,route,123784,"{'minlat': 48.8445799, 'minlon': 2.2299577, 'm...","[{'type': 'node', 'ref': 241928757, 'role': 's...","{'air_conditioning': 'yes', 'bicycle:condition...",yes,yes @ (Su 05:30-16:30),#F2C931,Château de Vincennes,...,Ligne1_RATP,no,Q13224,fr:Ligne 1 du métro de Paris,NaN,NaN,NaN,NaN,NaN,NaN
1,0,route,3328692,"{'minlat': 48.8444578, 'minlon': 2.2299297, 'm...","[{'type': 'node', 'ref': 1953056975, 'role': '...",{'bicycle:conditional': 'yes @ (Su 05:30-16:30...,NaN,yes @ (Su 05:30-16:30),#F2C931,La Défense,...,Ligne1_RATP,no,Q13224,fr:Ligne 1 du métro de Paris,NaN,NaN,NaN,NaN,NaN,NaN
2,0,route,4728208,"{'minlat': 48.8479623, 'minlon': 2.2777713, 'm...","[{'type': 'node', 'ref': 243496729, 'role': 's...","{'air_conditioning': 'yes', 'colour': '#216EB4...",yes,NaN,#216EB4,Porte Dauphine,...,Ligne2_RATP,limited,Q50718,fr:Ligne 2 du métro de Paris,25,NaN,NaN,NaN,NaN,NaN
3,0,route,123917,"{'minlat': 48.8479623, 'minlon': 2.2777713, 'm...","[{'type': 'node', 'ref': 4317605697, 'role': '...","{'air_conditioning': 'yes', 'colour': '#216EB4...",yes,NaN,#216EB4,Nation,...,Ligne2_RATP,no,Q50718,fr:Ligne 2 du métro de Paris,25,NaN,NaN,NaN,NaN,NaN
4,0,route,7731250,"{'minlat': 48.8629423, 'minlon': 2.2803973, 'm...","[{'type': 'node', 'ref': 7658079724, 'role': '...","{'colour': '#9A9940', 'from': 'Gallieni', 'nam...",NaN,NaN,#9A9940,Gallieni,...,Ligne3_RATP,no,Q50741,fr:Ligne 3 du métro de Paris,25,NaN,NaN,NaN,NaN,NaN
5,0,route,123789,"{'minlat': 48.8629423, 'minlon': 2.2803973, 'm...","[{'type': 'node', 'ref': 255161921, 'role': 's...","{'colour': '#9A9940', 'from': 'Pont de Levallo...",NaN,NaN,#9A9940,Pont de Levallois,...,NaN,no,Q50741,NaN,25,NaN,NaN,NaN,NaN,NaN
6,0,route,5456573,"{'minlat': 48.8651235, 'minlon': 2.3985462, 'm...","[{'type': 'node', 'ref': 4058743781, 'role': '...","{'colour': '#89C7D6', 'from': 'Gambetta', 'nam...",NaN,NaN,#89C7D6,Gambetta,...,NaN,no,Q50742,fr:Ligne 3 bis du métro de Paris,NaN,opendata RATP,NaN,NaN,NaN,NaN
7,0,route,123802,"{'minlat': 48.8651235, 'minlon': 2.3985462, 'm...","[{'type': 'node', 'ref': 1997626380, 'role': '...","{'colour': '#89C7D6', 'from': 'Porte des Lilas...",NaN,NaN,#89C7D6,Porte des Lilas,...,Ligne3_RATP,no,Q50742,fr:Ligne 3 bis du métro de Paris,NaN,opendata RATP,NaN,NaN,NaN,NaN
8,0,route,123767,"{'minlat': 48.8166247, 'minlon': 2.3182394, 'm...","[{'type': 'node', 'ref': 1780486821, 'role': '...","{'colour': '#BB4D98', 'from': 'Porte de Cligna...",NaN,NaN,#BB4D98,Porte de Clignancourt,...,Ligne4_RATP,no,Q50743,fr:Ligne 4 du métro de Paris,NaN,NaN,NaN,NaN,NaN,NaN
9,0,route,3326844,"{'minlat': 48.8166247, 'minlon': 2.3182394, 'm...","[{'type': 'node', 'ref': 2188235051, 'role': '...","{'colour': '#BB4D98', 'from': 'Mairie de Montr...",NaN,NaN,#BB4D98,Mairie de Montrouge,...,Ligne4_RATP,no,Q50743,fr:Ligne 4 du métro de Paris,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
pd.DataFrame.from_records(df_metro_route['members'][0])

,type,ref,role,lat,lon,geometry
0,node,7560870397,stop,48.894827,2.313262,NaN
1,node,264491193,stop_entry_only,48.827033,2.366416,NaN
2,node,3146958062,stop,48.829929,2.376717,NaN
3,node,3146958064,stop,48.833548,2.385954,NaN
4,node,3146958061,stop,48.839981,2.379581,NaN
5,node,1656938775,stop,48.843401,2.373960,NaN
6,node,3146958063,stop,48.859775,2.346303,NaN
7,node,3146958067,stop,48.865687,2.334520,NaN
8,node,3146958066,stop,48.870408,2.326565,NaN
9,way,678701597,platform,NaN,NaN,"[{'lat': 48.870185, 'lon': 2.3273491}, {'lat':..."


In [221]:
pd.DataFrame.from_records(df_railways[['members']])

,members
0,"[{'type': 'node', 'ref': 241928757, 'role': 's..."
1,"[{'type': 'node', 'ref': 1953056975, 'role': '..."
2,"[{'type': 'node', 'ref': 243496729, 'role': 's..."
3,"[{'type': 'node', 'ref': 4317605697, 'role': '..."
4,"[{'type': 'node', 'ref': 7658079724, 'role': '..."
5,"[{'type': 'node', 'ref': 255161921, 'role': 's..."
6,"[{'type': 'node', 'ref': 4058743781, 'role': '..."
7,"[{'type': 'node', 'ref': 1997626380, 'role': '..."
8,"[{'type': 'node', 'ref': 1780486821, 'role': '..."
9,"[{'type': 'node', 'ref': 2188235051, 'role': '..."


In [118]:
df_railway_subway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.subway,area=area_Paris,output='geom')


In [120]:
df_railway_subway['line'] = df_railway_subway['line'].fillna('Service')

df_railway_subway['first_line'] = df_railway_subway['line'].apply(lambda x: str(x).split(';')[0])

KeyError: 'line'

In [116]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    
# create a plot of the subway system    
output_file("Paris_subway.html")

color_mapping = {'M1':'yellow',
                 'M2':'red',
                 'M3':'blue',
                 'M4':'green',
            'Service':'white',
            'Other':'white',
                }

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}
    
p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),
            ("key", "@key"),
            ("tag", "@tag"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]


# plot the railway lines    
for line in set(df_railway_subway['line']):

    #print(line)
    
    df_lines = df_railway_subway.loc[df_railway_subway['line']==line]
  
    xx = list()
    yy = list()
    dashes = list()

    railway_lines = line.replace(',', ';').split(';')
    
    for row in df_lines['geometry']:
    
        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(row)['lat']),
                                                 list(pd.DataFrame.from_records(row)['lon']))
    
        xx.append(xxx)
        yy.append(yyy)
        
        
        if len(xxx)%2 == 0:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2))
        else:
            dashes.append([5,5*(len(railway_lines)-1)]*int(len(xxx)/2-0.5)+[5])
            
    
    source = ColumnDataSource({'xs':xx,'ys':yy,'line_dash':dashes})
    
    if len(railway_lines) == 1:
        line_dash = 'solid'
    else:
        line_dash = 'line_dash'
             
    for cnt,railway_line in enumerate(railway_lines):

            railway_line = railway_line.strip()

            p.multi_line('xs',
                   'ys',
                   source=source,
                   line_width=2,
                   #legend_label=str(railway_line),
                   line_color = color_mapping[railway_line],
                   line_dash = line_dash,
                   line_dash_offset = cnt*5
                  )

    
    
# plot the stations and add a hover tool
map_df = df_station_subway[['id','lat','lon','key','tags','tag','name','line','colour','operator']].copy()

map_df["x"],map_df["y"] = TRANSFORM_4326_TO_3857.transform(map_df['lat'],map_df['lon'])  
map_df["coordinates"]=tuple(zip(map_df["lat"],map_df["lon"]))
map_df['tags'] = map_df['tags'].apply(str)
map_df['tag'] = map_df['tag'].apply(str)
map_df['key'] = map_df['key'].apply(str)
map_df['line'] = map_df['line'].fillna('Other')

stations = list()
for legend_group in set(map_df["line"]):
    
    source = ColumnDataSource(map_df.loc[map_df["line"]==legend_group])

    
    station_lines = legend_group.split(';')
        
    for cnt,station_line in enumerate(station_lines):
        
        if len(station_lines)>1:
            
            station = p.wedge(x="x", y="y",
                            source=source,
                            start_angle=(cnt-1)/len(station_lines)*2*3.142,
                            end_angle=cnt/len(station_lines)*2*3.1412,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        else:
        
            station = p.circle(x="x", y="y",
                            source=source,
                            radius=100,
                            line_width=1,
                            fill_color = color_mapping[station_line],
                            line_color = 'white',
                            legend_label=station_line
                            )
            
        stations.append(station)
    

p.add_tools(HoverTool(tooltips=tooltips, renderers=stations))

p.legend.click_policy="hide"
    
    
show(p)

KeyError: 'Métro 8'

In [122]:
df_railway_subway.columns

Index(['type', 'id', 'bounds', 'nodes', 'geometry', 'tags', 'members',
       'electrified', 'frequency', 'gauge', 'name', 'operator', 'railway',
       'source', 'voltage', 'layer', 'level', 'rubber_tires', 'oneway',
       'railway:preferred_direction', 'bridge', 'covered', 'tunnel',
       'wikidata', 'wikipedia', 'name:ru', 'old_name', 'old_operator',
       'start_date', 'bridge:structure', 'detail', 'railway:track_ref',
       'usage', 'bridge:name', 'railway:ballastless', 'maxspeed', 'name:fr',
       'tracks', 'ref', 'cutting', 'location', 'boundary',
       'political_division', 'service', 'height', 'min_height',
       'ref:FR:geofer', 'passenger_lines', 'ref:FR:RATP', 'description',
       'disused:railway', 'station', 'indoor', 'embankment', 'oneway_1',
       'opening_date', 'owner', 'note', 'demolished', 'end_date', 'disused',
       'network', 'official_name', 'public_transport',
       'public_transport:version', 'ref:FR:FANTOIR', 'ref:FR:STIF', 'subway',
       'type:R